In [ ]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
from torch.autograd import Variable
from PIL import Image, ImageFilter


In [ ]:
# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.image_dir = 'images128x128'  # Directory path for images
        self.image_paths = sorted([os.path.join(self.image_dir, f) for f in os.listdir(self.image_dir) if f.endswith('.png')])
        self.clahe = cv2.createCLAHE(clipLimit=4, tileGridSize=(8,8))


    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert('L')  # Convert to grayscale

        # Convert the PIL image to a NumPy array
        img_np = np.array(img)

        # Apply CLAHE using OpenCV
        img_clahe = self.clahe.apply(img_np)

        # Convert back to PIL image (if your transformations expect PIL images)
        img = Image.fromarray(img_clahe)

        # Extract label from filename
        label = int(os.path.basename(img_path).split('_')[1].split('.')[0])

        # Apply any additional transformations if specified
        if self.transform:
            img = self.transform(img)

        return img, label

In [ ]:
import cv2
import matplotlib.pyplot as plt

# Load image
image = cv2.imread('images128x128/image1_0.png', cv2.IMREAD_GRAYSCALE)

# Apply Histogram Equalization
equalized_image = cv2.equalizeHist(image)

# Apply CLAHE
clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8,8))
clahe_image = clahe.apply(image)

# Display CLAHE result
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Original Image")
plt.imshow(image, cmap='gray')
plt.subplot(1, 2, 2)
plt.title("CLAHE Image")
plt.imshow(clahe_image, cmap='gray')
plt.show()
# Apply Sobel operator
sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=5)  # Horizontal edges
sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=5)  # Vertical edges
sobel_combined = cv2.magnitude(sobelx, sobely)

# Display Sobel result
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title("Sobel X")
plt.imshow(sobelx, cmap='gray')
plt.subplot(1, 2, 2)
plt.title("Sobel Y")
plt.imshow(sobely, cmap='gray')
plt.show()
# Apply simple thresholding
ret, thresholded_image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)

# Create a sharpening kernel
kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])

# Apply the kernel to the image
sharpened_image = cv2.filter2D(image, -1, kernel)

# Display sharpened image
plt.title("Sharpened Image")
plt.imshow(sharpened_image, cmap='gray')
plt.show()


In [ ]:
dataset = CustomImageDataset()
dataset[0][0]

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Load your images
image_label_1 = cv2.imread('images128x128/image61_0.png', cv2.IMREAD_GRAYSCALE)
image_label_2 = cv2.imread('images128x128/imagemass2119_1.png', cv2.IMREAD_GRAYSCALE)
image_label_3 = cv2.imread('images128x128/imagepneumonia1111_2.png', cv2.IMREAD_GRAYSCALE)

# Create a 1x3 grid
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

# Display the images in each column
axes[0].imshow(image_label_1, cmap='gray')
axes[1].imshow(image_label_2, cmap='gray')
axes[2].imshow(image_label_3, cmap='gray')

# Remove axis ticks
for ax in axes:
    ax.axis('off')

# Add text labels below the images
axes[0].text(0.5, -0.1, 'No Diagnosis', fontsize=12, ha='center', transform=axes[0].transAxes)
axes[1].text(0.5, -0.1, 'Diagnosed: Mass', fontsize=12, ha='center', transform=axes[1].transAxes)
axes[2].text(0.5, -0.1, 'Diagnosed: Pneumonia', fontsize=12, ha='center', transform=axes[2].transAxes)

# Adjust layout
plt.tight_layout()

# Display the grid
plt.show()


In [ ]:
# Define transformations for 128x128 images
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize images to 128x128
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,))  # Normalize for grayscale images
])
dataset = CustomImageDataset(transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
# Discriminator model for 128x128 images
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)  # Assuming 10 classes

        self.model = nn.Sequential(
            nn.Linear(128 * 128 + 10, 1024),  # Input size is now 128*128 + 10
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x, labels):
        x = x.view(x.size(0), -1)  # Flatten the image tensor
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()


In [ ]:
# Generator model for 128x128 images
class Generator(nn.Module):
    def __init__(self):
        super().__init__()

        self.label_emb = nn.Embedding(10, 10)  # Assuming 10 classes

        self.model = nn.Sequential(
            nn.Linear(100 + 10, 256),  # Assuming noise vector size is 100
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 128 * 128),  # Adjust output size to 128*128
            nn.Tanh()  # Use Tanh to scale output to [-1, 1]
        )

    def forward(self, z, labels):
        z = z.view(z.size(0), 100)  # Ensure z is of size (batch_size, 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)  # Concatenate noise and label
        out = self.model(x)
        return out.view(z.size(0), 1, 128, 128)  # Reshape to (batch_size, 1, 128, 128)


In [ ]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()

In [ ]:
#DONT CHANGE

criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.0001)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.000005)

In [ ]:
#criterion = nn.BCELoss()
#d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=0.000005)
#g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.0001)

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion):
    g_optimizer.zero_grad()
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 10, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, Variable(torch.ones(batch_size)).cuda())
    g_loss.backward()
    g_optimizer.step()
    return g_loss.item()

In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # Train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)).cuda())

    # Train with fake images
    z = Variable(torch.randn(batch_size, 100)).cuda()
    fake_labels = Variable(torch.LongTensor(np.random.randint(0, 3, batch_size))).cuda()
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)).cuda())

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.item()

In [ ]:
num_epochs = 500
x = 0
for epoch in range(num_epochs):
    x += 1
    if(x%20 == 0):
        torch.save(generator.state_dict(), f'generator{x}.pth')
    print('Starting epoch {}...'.format(epoch))
    for i, (images, labels) in enumerate(data_loader):
        real_images = Variable(images).cuda()
        labels = Variable(labels).cuda()
        generator.train()
        batch_size = real_images.size(0)
        d_loss = discriminator_train_step(len(real_images), discriminator,
                                          generator, d_optimizer, criterion,
                                          real_images, labels)


        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

    generator.eval()
    print('g_loss: {}, d_loss: {}'.format(g_loss, d_loss))
    z = Variable(torch.randn(9, 100)).cuda()
    labels = Variable(torch.LongTensor(np.arange(9))).cuda()
    sample_images = generator(z, labels).data.cpu()
    grid = make_grid(sample_images, nrow=3, normalize=True).permute(1,2,0).numpy()
    plt.imshow(grid)
    plt.show()
